In [1]:
import os
from typing import Tuple, List, Callable, Iterator, Optional, Dict, Any
from collections import defaultdict

In [2]:
import albumentations as A
from albumentations.pytorch import ToTensorV2
import tensorboardX
from tensorboardX import SummaryWriter
import os

import pandas as pd 
import numpy as np
import glob
from tqdm import tqdm
import cv2
from sklearn.model_selection import train_test_split
import fiona
import segmentation_models_pytorch as smp

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset
from torchvision import datasets, models, transforms
from torchvision.models import resnet18
from torchvision.utils import draw_segmentation_masks
import torchvision.transforms as T

from PIL import Image
import matplotlib.pyplot as plt
from IPython.display import clear_output

import warnings
warnings.filterwarnings("ignore")

import json
import secrets

In [3]:
torch.manual_seed(0)
np.random.seed(0)

In [4]:
from unet import *
from unetpp import *

In [5]:
class EyeDataset(Dataset):

    def __init__(self, size, data_folder: str, transform = None, grayscale=False):
        self.data_folder = data_folder
        self.transform = transform
        self.grayscale = grayscale
        
        self._image_files = glob.glob(os.path.join(data_folder, "images/*.png"))
        self._masks_files = os.path.join(data_folder, 'new_masks')
        
        self._images, self._masks = self._load()
        
        self.image_size = size
        
        
    @staticmethod
    def read_image(path: str) -> np.ndarray:
        image = cv2.imread(str(path), cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = np.array(image / 255, dtype=np.float32)

        return image
    
    def _load(self):
        images = []
        masks = []
        for img in self._image_files:
            name = img.split("/")[-1].split(".")[0]
            image = None
            
            json_path = os.path.join(self._masks_files, name + ".npy")
            
            mask = np.load(json_path)
            mask = np.clip(mask, 0, 1)
            
            images.append(image)
            masks.append(mask)
            
        return images, masks
    
        
    def __getitem__(self, idx: int) -> dict:
        
        img = self._image_files[idx]
        image = self.read_image(img)
        
        if self.grayscale:
            image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            image = np.expand_dims(image, axis=-1)
        
        mask = self._masks[idx]
        
        
        if self.image_size != None:
            image = cv2.resize(image, self.image_size, interpolation = cv2.INTER_CUBIC)
            mask = cv2.resize(mask, self.image_size, interpolation = cv2.INTER_CUBIC)
        
        mask = np.expand_dims(mask, axis=-1)
        sample = {'image': image,
                  'mask': mask}

        if self.transform is not None:
            sample = self.transform(**sample)
        return sample

    def __len__(self):
        return len(self._image_files)


class DatasetPart(Dataset):
    """
    Обертка над классом датасета для его разбиения на части
    """
    def __init__(self, dataset: Dataset,
                 indices: np.ndarray,
                 transform: A.Compose = None):
        self.dataset = dataset
        self.indices = indices

        self.transform = transform

    def __getitem__(self, idx: int) -> dict:
        sample = self.dataset[self.indices[idx]]

        if self.transform is not None:
            sample = self.transform(**sample)

        return sample

    def __len__(self) -> int:
        return len(self.indices)

In [6]:
IMAGE_SHAPE = (1624, 1232)
SCALE = 2

In [7]:
grayscale = False
#size = (IMAGE_SHAPE[0] // SCALE, IMAGE_SHAPE[1] // SCALE)
size = (1024, 1024)
dataset = EyeDataset(size=size, data_folder="../train_dataset_mc", grayscale=grayscale)

In [8]:
train_list = A.Compose(
                    [#A.RandomCrop(width=1024, height=1024),
                     A.ShiftScaleRotate(scale_limit=[-0.2, 0.22], rotate_limit=90, interpolation=cv2.INTER_CUBIC,
                                border_mode=cv2.BORDER_REFLECT_101, p=0.6),
                     A.HorizontalFlip(p=0.5),
                     A.VerticalFlip(p=0.5), 
                    A.HueSaturationValue(hue_shift_limit=10 / 255, sat_shift_limit=10 / 255,\
                                            val_shift_limit=20 / 255, always_apply=False, p=0.2),
                    A.RandomBrightnessContrast(p=0.2),
                    A.ChannelShuffle(p=0.2),
                     ToTensorV2(transpose_mask=True)
                    ],
                )
eval_list = A.Compose(
                    [#A.RandomCrop(width=1024, height=1024),
                     A.ShiftScaleRotate(scale_limit=[-0.65, 0.25], rotate_limit=90, interpolation=cv2.INTER_CUBIC,
                                border_mode=cv2.BORDER_REFLECT_101, p=0.6),
                     A.HorizontalFlip(p=0.5),
                     A.VerticalFlip(p=0.5),
                    #A.OneOf([
                    #         A.HueSaturationValue(hue_shift_limit=10 / 255, sat_shift_limit=10 / 255,\
                    #                              val_shift_limit=20 / 255, always_apply=False, p=0.2),
                    #         A.RandomBrightnessContrast(p=0.2),
                    #         A.ChannelShuffle(p=0.2),
                    #], p=0.5),
                     ToTensorV2(transpose_mask=True)
                    ],
                )
#eval_list = [#A.LongestMaxSize(size, interpolation=cv2.INTER_CUBIC),
              #A.PadIfNeeded(size, size),
#              ToTensorV2(transpose_mask=True)
#            ]

transforms = {'train': train_list, 'test': A.Compose(eval_list)}


In [9]:
train_indices, test_indices = train_test_split(range(len(dataset)), test_size=0.2)


In [10]:
train_dataset = DatasetPart(dataset, train_indices, transform=transforms['train'] )
valid_dataset = DatasetPart(dataset, test_indices, transform=transforms['test'] )


In [11]:
batch_size=2

In [12]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size,
                                   num_workers=4,
                                   shuffle=True, drop_last=True)

valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size,
                                   num_workers=4,
                                   shuffle=True, drop_last=True)

In [13]:
class SoftDice:
    def __init__(self, epsilon=1e-8):
        self.epsilon = epsilon

    def __call__(self, predictions: List[Dict[str, torch.Tensor]],
                 targets: List[Dict[str, torch.Tensor]]) -> torch.Tensor:
        
        tags = torch.where(predictions > 0.5, 1, 0)
        numerator = torch.sum(2 * tags * targets)
        denominator = torch.sum(tags + targets)
        return (numerator + self.epsilon) / (denominator + self.epsilon)

# Метрика полноты
class Recall:
    def __init__(self, epsilon=1e-8, b=1):
        self.epsilon = epsilon
        self.a = b*b

    def __call__(self, predictions: List[Dict[str, torch.Tensor]],
                 targets: List[Dict[str, torch.Tensor]]) -> torch.Tensor:
        
        tags = torch.where(predictions > 0.5, 1, 0)
        numerator = torch.sum(tags * targets)
        denominator = torch.sum(targets)

        return numerator / (denominator + self.epsilon)

# Метрика точности
class Accuracy:
    def __init__(self, epsilon=1e-8, b=1):
        self.epsilon = epsilon
        self.a = b*b

    def __call__(self, predictions: list, targets: list) -> torch.Tensor:
        tags = torch.where(predictions > 0.5, 1, 0)
        corrects = (tags == targets).float()
        acc = corrects.sum()/corrects.numel()
        return acc

def make_metrics():
    soft_dice = SoftDice()
    rec = Recall()
    acc = Accuracy()

    def dice(pred, target):
        return soft_dice(pred, target)

    def accuracy(pred, target):
        return acc(pred, target)

    def recall(pred, target):
        return rec(pred, target)

    return [('exp_dice', dice),
            ('accuracy', accuracy),
            ('recall', recall),
            ]

In [14]:
model_type='manet'
##model = smp.Unet(model_type, activation='sigmoid', classes=1).cuda()
#model = smp.UnetPlusPlus(encoder_name='resnet34', encoder_depth=5, encoder_weights='imagenet', in_channels=3, classes=1, activation='sigmoid')
model = smp.MAnet(encoder_name='resnet34', encoder_depth=5, encoder_weights='imagenet', in_channels=3, classes=1, activation='sigmoid')
#model = ResNet34UnetPlus(3, 1)
#model = UNet_classic(n_channels=3, n_classes=1, bilinear=True) #n_classes = 3
#model = UNet_RCAB(3, 1)
#model = UNet_reduced(n_channels=3, n_classes=1)
model = model.cuda()

In [15]:
criterion = smp.losses.DiceLoss(smp.losses.BINARY_MODE, from_logits=False)
bce_loss = nn.BCELoss().cuda()

In [16]:
learning_rate = 1e-3
optimizer = torch.optim.Adam(model.parameters(), learning_rate)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.98)

In [17]:
metrics = make_metrics()

In [18]:
def train_epoch(train_loader, model, metrics, optimizer, criterion):
    model.train()
    
    total_loss = 0.
    i = 0
    
    summary = {"loss" : 0.}
    
    for m, f in metrics:
        summary[m] = 0.
        
    for batch in tqdm(iter(train_loader)):
        optimizer.zero_grad()
        i += 1
        image = batch['image'].cuda()
        mask_target = batch["mask"].cuda()

        mask_pred = model(image)

        loss =  bce_loss(mask_pred, mask_target) + 0.5*criterion(mask_pred, mask_target) 
        loss.backward()
        optimizer.step()
        
        summary["loss"] += loss.item()
        
        for metric_name, metric_fn in metrics:
            summary[metric_name] += metric_fn(mask_pred, mask_target).item()
    for m in summary.keys():
        summary[m] = summary[m]/i
    return summary


In [19]:
def val_epoch(val_loader, model, metrics, criterion):
        
    model.eval()
    
    total_loss = 0.
    i = 0
    
    summary = {"loss" : 0.}
    
    for m, f in metrics:
        summary[m] = 0.
        
    for batch in tqdm(iter(val_loader)):
        i += 1
        image = batch['image'].cuda()
        mask_target = batch["mask"].cuda()
        
        mask_pred = model(image)
        loss = criterion(mask_pred, mask_target)
        
        summary["loss"] += loss.item()
        
        for metric_name, metric_fn in metrics:
            summary[metric_name] += metric_fn(mask_pred, mask_target).item()
            

    for m in summary.keys():
        summary[m] = summary[m]/i

    return summary

In [22]:
model_type='manet'
experiment_name = str(model_type) + "_" + secrets.token_hex(nbytes=8)
log_dir = './logs'
weights_dir = "./weights"
writer = SummaryWriter(os.path.join(log_dir, experiment_name))

In [23]:
print(experiment_name)

manet_ad2ca560d36a45d2


In [24]:
if not os.path.exists(weights_dir):
    os.mkdir(weights_dir)

cur_experiment_weights = os.path.join(weights_dir, experiment_name)

if not os.path.exists(cur_experiment_weights):
    os.mkdir(cur_experiment_weights)

In [25]:
epochs = 400
save_every = 10

In [26]:
for epoch in range(epochs):
    summary = train_epoch(train_loader, model, metrics, optimizer, criterion)
    print("Train", summary)
    scheduler.step()
    for metric_name in summary.keys():
        writer.add_scalar('Metrics_train/{}'.format(metric_name), summary[metric_name], epoch)
        
    with torch.no_grad():
        val_summary = val_epoch(valid_loader, model, metrics, criterion)
        print("Test", val_summary)
        for metric_name in val_summary.keys():
            writer.add_scalar('Metrics_val/{}'.format(metric_name), summary[metric_name], epoch)
            
    if (epoch + 1) % save_every == 0:
        save_dir = os.path.join(cur_experiment_weights, "{}_{}_weights.pth".format(epoch, val_summary["exp_dice"]))
        torch.save(model.state_dict(), save_dir)
        
save_dir = os.path.join(cur_experiment_weights, "weights_last.pth")
torch.save(model.state_dict(), save_dir)

100%|█████████████████████████████████████████████| 260/260 [00:38<00:00,  6.77it/s]

Train {'loss': 0.39950602920009537, 'exp_dice': 0.3850264413666081, 'accuracy': 0.9713907993756807, 'recall': 0.44910871771128136}



100%|███████████████████████████████████████████████| 65/65 [00:03<00:00, 17.19it/s]

Test {'loss': 0.6191488513579735, 'exp_dice': 0.40619650322657364, 'accuracy': 0.9768554247342623, 'recall': 0.3318966880440712}



100%|█████████████████████████████████████████████| 260/260 [00:36<00:00,  7.06it/s]

...

Test {'loss': 0.43557012447944055, 'exp_dice': 0.5808251674358661, 'accuracy': 0.9765846105722281, 'recall': 0.6286132209576093}



100%|█████████████████████████████████████████████| 260/260 [00:35<00:00,  7.30it/s]

Train {'loss': 0.2320260633642857, 'exp_dice': 0.6213820211589336, 'accuracy': 0.9751903992432814, 'recall': 0.6665845312751256}



100%|███████████████████████████████████████████████| 65/65 [00:03<00:00, 17.84it/s]


Test {'loss': 0.42780311015936046, 'exp_dice': 0.5892573934334975, 'accuracy': 0.9759622720571665, 'recall': 0.6388923823833466}


In [27]:
test_dataset_path = "../test_dataset_mc2/eye_test"
test_imgs = glob.glob(test_dataset_path + '/*.png')
test_imgs = glob.glob(test_dataset_path + '/*.png')

In [28]:
def read_image(size, path: str) -> np.ndarray:
    image = cv2.imread(str(path), cv2.IMREAD_COLOR)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = np.array(image / 255, dtype=np.float32)
    print(image.shape)
    #image = cv2.resize(image, (size, size), interpolation = cv2.INTER_AREA)

    return image

def read_image_gray(size, path: str) -> np.ndarray:
    image = cv2.imread(str(path), cv2.IMREAD_COLOR)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image = np.array(image / 255, dtype=np.float32)
    image = np.expand_dims(image, -1)
    #image = cv2.resize(image, (size, size), interpolation = cv2.INTER_AREA)

    return image

In [29]:
test_pics = glob.glob("./sample_solution_old/*.png")

In [26]:
save_dir = os.path.join('./weights/unetpp_standart_0096a2d74ceb745e', "weights_last.pth")
print(save_dir)
#torch.save(model.state_dict(), save_dir)
#model.load_state_dict(torch.load(save_dir))

./weights/unetpp_standart_0096a2d74ceb745e/weights_last.pth


<All keys matched successfully>

In [29]:
model.eval()

MAnet(
  (encoder): ResNetEncoder(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, trac

In [30]:
import torchvision.transforms as T
from PIL import Image
import PIL.ImageOps
transform = T.ToPILImage()

In [ ]:
save_probs = False

In [32]:
for img in test_imgs: #pics
    name = img.split("/")[-1].split(".")[0]
    input_path = os.path.join(test_dataset_path, name + ".png")
    
    cur_image = read_image(512, input_path)
    im = cv2.resize(cur_image, (1024,1024), interpolation = cv2.INTER_CUBIC) #(512, 512)
    image = np.expand_dims(im, axis=0)
    image = np.transpose(image, (0, 3, 1, 2))
    input = torch.tensor(image).cuda()
    mask = model(input)
    out = mask.cpu().detach().numpy()[0]
    out = np.transpose(out, (1, 2, 0))
    out = cv2.resize(out, (1624, 1232), interpolation = cv2.INTER_CUBIC)
    if save_probs:
        out = np.array(out*255, dtype='uint8')
        np.save(os.path.join('./manet_ad2ca560d36a45d2_probs', name + ".npy"), out)
    else:
        out = np.array(np.where(out >0.5, 1, 0)*255, dtype='uint8')
        cv2.imwrite(os.path.join('./manet_ad2ca560d36a45d2', name + ".png"), out)

(1232, 1624, 3)
(1232, 1624, 3)
(1232, 1624, 3)
(1232, 1624, 3)
(1232, 1624, 3)
(1232, 1624, 3)
(1232, 1624, 3)
(1232, 1624, 3)
(1232, 1624, 3)
(1232, 1624, 3)
(1232, 1624, 3)
(1232, 1624, 3)
(1232, 1624, 3)
(1232, 1624, 3)
(1232, 1624, 3)
(1232, 1624, 3)
(1232, 1624, 3)
(1232, 1624, 3)
(1232, 1624, 3)
(1232, 1624, 3)
(1232, 1624, 3)
(1232, 1624, 3)
(1232, 1624, 3)
(1232, 1624, 3)
(1232, 1624, 3)
(1232, 1624, 3)
(1232, 1624, 3)
(1232, 1624, 3)
(1232, 1624, 3)
(1232, 1624, 3)
(1232, 1624, 3)
(1232, 1624, 3)
(1232, 1624, 3)
(1232, 1624, 3)
(1232, 1624, 3)
(1232, 1624, 3)
(1232, 1624, 3)
(1232, 1624, 3)
(1232, 1624, 3)
(1232, 1624, 3)
(1232, 1624, 3)
(1232, 1624, 3)
(1232, 1624, 3)
(1232, 1624, 3)
(1232, 1624, 3)
(1232, 1624, 3)
(1232, 1624, 3)
(1232, 1624, 3)
(1232, 1624, 3)
(1232, 1624, 3)
(1232, 1624, 3)
(1232, 1624, 3)
(1232, 1624, 3)
(1232, 1624, 3)
(1232, 1624, 3)
(1232, 1624, 3)
(1232, 1624, 3)
(1232, 1624, 3)
(1232, 1624, 3)
(1232, 1624, 3)
(1232, 1624, 3)
(1232, 1624, 3)
(1232, 1